In [6]:
import pyarrow.parquet as pq
import polars as pl
# Paths to your files
quotes_path = '~/Documents/TAQData/data_quotes_2024_03_15.parquet'
trades_path = '~/Documents/TAQData/data_trades_2024_03_15.parquet'

# Efficiently count rows without loading data
quotes_rows = pq.ParquetFile(quotes_path).metadata.num_rows
trades_rows = pq.ParquetFile(trades_path).metadata.num_rows

print(f"Quotes rows: {quotes_rows:,}")
print(f"Trades rows: {trades_rows:,}")

Quotes rows: 1,665,669,705
Trades rows: 82,461,761


In [24]:
import pyarrow.parquet as pq
from datetime import datetime
# ---- quotes file ----
quotes_path = '~/Documents/TAQData/data_quotes_2024_03_15.parquet'
pf_quotes = pq.ParquetFile(quotes_path)
print("🧾 Quotes schema:")
print(pf_quotes.schema_arrow)     # column names and types only

# ---- trades file ----
trades_path = '~/Documents/TAQData/data_trades_2024_03_15.parquet'
pf_trades = pq.ParquetFile(trades_path)
print("\n💹 Trades schema:")
print(pf_trades.schema_arrow)

🧾 Quotes schema:
DATE: date32[day]
TIME_M: time64[us]
EX: string
BID: double
BIDSIZ: int64
ASK: double
ASKSIZ: int64
QU_COND: string
QU_SEQNUM: int64
NATBBO_IND: string
QU_CANCEL: string
QU_SOURCE: string
SYM_ROOT: string
SYM_SUFFIX: string

💹 Trades schema:
DATE: date32[day]
TIME_M: time64[us]
EX: string
SYM_ROOT: string
SYM_SUFFIX: string
TR_SCOND: string
SIZE: int64
PRICE: double
TR_STOP_IND: string
TR_CORR: string
TR_SEQNUM: int64
TR_ID: int64
TR_SOURCE: string
TR_RF: string


In [81]:
#Trades
#%run polar_try_partitioned.py --FILE_PATH=/home/amazon/Documents/TAQData/data_trades_2024_03_15.parquet --OUTPUT_DIR=/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades/

In [82]:
#Quotes
#%run polar_try_partitioned.py --FILE_PATH=/home/amazon/Documents/TAQData/data_quotes_2024_03_15.parquet --OUTPUT_DIR=/home/amazon/Documents/TAQData/2024_03_15/processed_output_quotes/

In [23]:
pl.Config.set_tbl_cols(50) 
pl.Config.set_tbl_width_chars(200)
pl.Config.set_tbl_rows(200) #polars.Config.tbl_rows = 50

polars.config.Config

In [8]:
#%run makesTradesColsUpper.py --INPUT="/home/amazon/Documents/TAQData/data_trades_2024_03_15.parquet" --OUTPUT="/home/amazon/Documents/TAQData/data_trades_2024_03_15_upper.parquet"
trades_path = '~/Documents/TAQData/data_trades_2024_03_15_upper.parquet'
pf_trades = pq.ParquetFile(trades_path)
print("\n💹 Trades schema:")
print(pf_trades.schema_arrow)


💹 Trades schema:
DATE: date32[day]
TIME_M: time64[us]
EX: string
SYM_ROOT: string
SYM_SUFFIX: string
TR_SCOND: string
SIZE: int64
PRICE: double
TR_STOP_IND: string
TR_CORR: string
TR_SEQNUM: int64
TR_ID: int64
TR_SOURCE: string
TR_RF: string


In [9]:
#%run rewrite2.py --IN_FILE="/home/amazon/Documents/TAQData/data_trades_2024_03_15_upper.parquet" --OUT_DIR="/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper_clean_from_single/" --BATCH_ROWS=25000
from pathlib import Path
import pyarrow.parquet as pq

DIR = Path("/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper_clean_from_single/")

files = sorted(DIR.glob("*.parquet"))
if not files:
    raise RuntimeError(f"No parquet files found in {DIR}")

for f in files[:1]:
    arrow_schema = pq.read_schema(f)   # Arrow schema
    print(f"{f.name}")
    print(f"{arrow_schema}")


chunk_000001.parquet
DATE: date32[day]
TIME_M: time64[us]
EX: string
SYM_ROOT: string
SYM_SUFFIX: string
TR_SCOND: string
SIZE: int64
PRICE: double
TR_STOP_IND: string
TR_CORR: string
TR_SEQNUM: int64
TR_ID: int64
TR_SOURCE: string
TR_RF: string


In [219]:
#%run query.py  --TRADES_UPPER="/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper/*.parquet" --QUOTES_OLD="/home/amazon/Documents/TAQData/2024_03_15/processed_output_quotes/*.parquet" --OUTPUT_FILE="/home/amazon/Documents/TAQData/2024_03_15/taq_analysis_output.txt"

In [10]:
import polars as pl
QUOTES="/home/amazon/Documents/TAQData/data_quotes_2024_03_15.parquet"
q= pl.scan_parquet(QUOTES).filter((pl.col("SYM_ROOT") == "SPY") &(pl.col("SYM_SUFFIX").fill_null("") == "")).collect()
len(q)

27516767

In [11]:
QUOTES_DIR="/home/amazon/Documents/TAQData/2024_03_15/processed_output_quotes/"
q= pl.scan_parquet(f"{QUOTES_DIR}/*.parquet").filter((pl.col("SYM_ROOT") == "SPY") &(pl.col("SYM_SUFFIX").fill_null("") == "")).collect()
len(q)

27516767

In [12]:
QUOTES_DIR="/home/amazon/Documents/TAQData/2024_03_15/processed_output_quotes_top50/"
q= pl.scan_parquet(f"{QUOTES_DIR}/*.parquet").filter((pl.col("SYM_ROOT") == "SPY") &(pl.col("SYM_SUFFIX").fill_null("") == "")).collect()
len(q)

27516767

In [13]:
TRADES="/home/amazon/Documents/TAQData/data_trades_2024_03_15.parquet"
t= pl.scan_parquet(TRADES).filter((pl.col("SYM_ROOT") == "SPY") &(pl.col("SYM_SUFFIX").fill_null("") == "")).collect()
len(t)

694399

In [14]:
TRADES="/home/amazon/Documents/TAQData/data_trades_2024_03_15_upper.parquet"
t= pl.scan_parquet(TRADES).filter((pl.col("SYM_ROOT") == "SPY") &(pl.col("SYM_SUFFIX").fill_null("") == "")).collect()
len(t)

694399

In [163]:
# TRADES_DIR="/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades/"
# q= pl.scan_parquet(f"{TRADES_DIR}/*.parquet").filter((pl.col("SYM_ROOT") == "SPY") &(pl.col("SYM_SUFFIX").fill_null("") == "")).collect()
# len(q)

In [18]:
import polars as pl

schema = {
    "DATE": pl.Date,
    "TIME_M": pl.Time,
    "EX": pl.Utf8,
    "SYM_ROOT": pl.Utf8,
    "SYM_SUFFIX": pl.Utf8,
    "TR_SCOND": pl.Utf8,
    "SIZE": pl.Int64,
    "PRICE": pl.Float64,
    "TR_STOP_IND": pl.Utf8,
    "TR_CORR": pl.Utf8,
    "TR_SEQNUM": pl.Int64,
    "TR_ID": pl.Int64,
    "TR_SOURCE": pl.Utf8,
    "TR_RF": pl.Utf8,
}

TRADES="/home/amazon/Documents/TAQData/data_trades_2024_03_15_upper.parquet"
t= pl.scan_parquet(TRADES, schema=schema).filter((pl.col("SYM_ROOT") == "SPY") &(pl.col("SYM_SUFFIX").fill_null("") == "")).collect()
print(len(t))
t= pl.scan_parquet(TRADES, schema=schema).collect()
print(len(t))

694399
82461761


In [20]:
import polars as pl

schema = {
    "DATE": pl.Date,
    "TIME_M": pl.Time,
    "EX": pl.Utf8,
    "SYM_ROOT": pl.Utf8,
    "SYM_SUFFIX": pl.Utf8,
    "TR_SCOND": pl.Utf8,
    "SIZE": pl.Int64,
    "PRICE": pl.Float64,
    "TR_STOP_IND": pl.Utf8,
    "TR_CORR": pl.Utf8,
    "TR_SEQNUM": pl.Int64,
    "TR_ID": pl.Int64,
    "TR_SOURCE": pl.Utf8,
    "TR_RF": pl.Utf8,
}

TRADES="/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper_clean/"
t= pl.scan_parquet(TRADES, schema=schema).filter((pl.col("SYM_ROOT") == "SPY") &(pl.col("SYM_SUFFIX").fill_null("") == "")).collect()
t= pl.scan_parquet(TRADES).filter((pl.col("SYM_ROOT") == "SPY") &(pl.col("SYM_SUFFIX").fill_null("") == "")).collect()
print(len(t))
TRADES="/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper_clean_from_single/"
t= pl.scan_parquet(TRADES).filter((pl.col("SYM_ROOT") == "SPY") &(pl.col("SYM_SUFFIX").fill_null("") == "")).collect()
print(len(t))
TRADES="/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper_clean_from_single/"
t= pl.scan_parquet(TRADES).collect()
print(len(t))

1388798
694399
82461761


In [21]:
%run offending_datatypes.py --TRADES_DIR=/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper_clean_from_single

Found 3299 parquet files in /home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper_clean_from_single
  scanned schemas: 200/3299
  scanned schemas: 400/3299
  scanned schemas: 600/3299
  scanned schemas: 800/3299
  scanned schemas: 1000/3299
  scanned schemas: 1200/3299
  scanned schemas: 1400/3299
  scanned schemas: 1600/3299
  scanned schemas: 1800/3299
  scanned schemas: 2000/3299
  scanned schemas: 2200/3299
  scanned schemas: 2400/3299
  scanned schemas: 2600/3299
  scanned schemas: 2800/3299
  scanned schemas: 3000/3299
  scanned schemas: 3200/3299
  scanned schemas: 3299/3299

=== Columns with >1 dtype across files ===

=== SIZE dtype breakdown ===
SIZE dtype Int64: 3299 files
  examples: ['chunk_000001.parquet', 'chunk_000002.parquet', 'chunk_000003.parquet', 'chunk_000004.parquet', 'chunk_000005.parquet', 'chunk_000006.parquet', 'chunk_000007.parquet', 'chunk_000008.parquet', 'chunk_000009.parquet', 'chunk_000010.parquet']

=== Offending files (SIZE != Int64) 

In [22]:
#%run query.py  --TRADES_UPPER="/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper_clean_from_single/*.parquet" --QUOTES_OLD="/home/amazon/Documents/TAQData/2024_03_15/processed_output_quotes/*.parquet" --OUTPUT_FILE="/home/amazon/Documents/TAQData/2024_03_15/taq_analysis_output.txt"

In [28]:
#%run stats.py --TRADES_DIR="/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper_clean_from_single/" --QUOTES_DIR="/home/amazon/Documents/TAQData/2024_03_15/processed_output_quotes/" --OUT_DIR="/home/amazon/Documents/TAQData/2024_03_15/stats_out_simple/"

In [87]:
OUT_DIR="/home/amazon/Documents/TAQData/2024_03_15/stats_out_simple"
TRADES_OUT_CSV = f"{OUT_DIR}/top50_trades_by_volume.csv"
QUOTES_OUT_CSV = f"{OUT_DIR}/top50_quotes_by_rows.csv"

print(f"Reading {TRADES_OUT_CSV} and {QUOTES_OUT_CSV} …")
trades_df = pl.read_csv(TRADES_OUT_CSV)
quotes_df = pl.read_csv(QUOTES_OUT_CSV)

print ("Top Trades")
print(trades_df.sort("TRADES_ROWS", descending=True))
print ("Top Quotes")
print(quotes_df.sort("QUOTES_ROWS", descending=True))

Reading /home/amazon/Documents/TAQData/2024_03_15/stats_out_simple/top50_trades_by_volume.csv and /home/amazon/Documents/TAQData/2024_03_15/stats_out_simple/top50_quotes_by_rows.csv …
Top Trades
shape: (50, 4)
┌──────────┬────────────┬─────────────┬────────────┐
│ SYM_ROOT ┆ SYM_SUFFIX ┆ TRADES_ROWS ┆ TRADES_VOL │
│ ---      ┆ ---        ┆ ---         ┆ ---        │
│ str      ┆ str        ┆ i64         ┆ i64        │
╞══════════╪════════════╪═════════════╪════════════╡
│ TSLA     ┆            ┆ 1107061     ┆ 107574931  │
│ AAPL     ┆            ┆ 771409      ┆ 162289433  │
│ AMD      ┆            ┆ 726550      ┆ 107720633  │
│ VERB     ┆            ┆ 723088      ┆ 546980770  │
│ SPY      ┆            ┆ 694399      ┆ 117996619  │
│ AMZN     ┆            ┆ 465377      ┆ 96036298   │
│ XOM      ┆            ┆ 375619      ┆ 201011675  │
│ WFC      ┆            ┆ 330123      ┆ 203186684  │
│ SOUN     ┆            ┆ 327723      ┆ 130961335  │
│ JTAI     ┆            ┆ 296601      ┆ 10169782

In [24]:
#%run top50_trades_corresponding_quotes.py --OUT_DIR="/home/amazon/Documents/TAQData/2024_03_15/stats_out_simple/" --TRADES_TOP50_CSV=top50_trades_by_volume.csv --TRADES_SRC_DIR="/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper/" --TRADES_OUT_DIR="/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper_top50/" --QUOTES_SRC_DIR="/home/amazon/Documents/TAQData/2024_03_15/processed_output_quotes/" --QUOTES_OUT_DIR="/home/amazon/Documents/TAQData/2024_03_15/processed_output_quotes_top50/" --MERGED_OUT_CSV="merged_top50_sorted.csv"

In [202]:
#%run top50_trades_corresponding_quotes_persist.py --TRADES_DIR="/home/amazon/Documents/TAQData/2024_03_15/processed_output_trades_upper_top50/" --QUOTES_DIR="/home/amazon/Documents/TAQData/2024_03_15/processed_output_quotes_top50/" --STATS_DIR="/home/amazon/Documents/TAQData/2024_03_15/stats_out_simple/" --TOP50_CSV="top50_trades_by_volume.csv" --OUT_DIR="/home/amazon/Documents/TAQData/2024_03_15/merged_output_top50/" --OUT_FILE="merged_all.parquet" --CHUNK_SIZE=25000

In [46]:
%run ms.py --MERGED_FILE="/home/amazon/Documents/TAQData/2024_03_15/merged_output_top50/merged_all.parquet" --OUT_DIR="/home/amazon/Documents/TAQData/2024_03_15/ms_out/" --TICKERS=VERB,FSR,BMY

Null TR_SEQNUM : 0
Null QU_SEQNUM : 206422
Zero BID and ASK : 15134

🔎 Running microstructure analysis for: VERB
shape: (1, 19)
┌────────┬────────┬────────────┬──────────┬─────────────┬─────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ TICKER ┆ N_ROWS ┆ TOTAL_SIZE ┆ VWAP     ┆ AVG_SPREAD_ ┆ AVG_DEPTH_T ┆ IS_MEAN_BP ┆ IS_MEDIAN_ ┆ IS_MIN_BPS ┆ IS_MAX_BPS ┆ MI_MEAN_BP ┆ MI_MEDIAN_ ┆ MI_MIN_BPS ┆ MI_MAX_BPS ┆ MR_MEAN_BP ┆ MR_MEDIAN_ ┆ MR_MIN_BPS ┆ MR_MAX_BPS ┆ FWD_TRADES │
│ ---    ┆ ---    ┆ ---        ┆ ---      ┆ BPS         ┆ OB          ┆ S          ┆ BPS        ┆ ---        ┆ ---        ┆ S          ┆ BPS        ┆ ---        ┆ ---        ┆ S          ┆ BPS        ┆ ---        ┆ ---        ┆ _FOR_MR    │
│ str    ┆ u32    ┆ i64        ┆ f64      ┆ ---         ┆ ---         ┆ ---        ┆ ---        ┆ f64        ┆ f64        ┆ ---        ┆ ---        ┆

In [47]:
summary_df

TICKER,N_ROWS,TOTAL_SIZE,VWAP,AVG_SPREAD_BPS,AVG_DEPTH_TOB,IS_MEAN_BPS,IS_MEDIAN_BPS,IS_MIN_BPS,IS_MAX_BPS,MI_MEAN_BPS,MI_MEDIAN_BPS,MI_MIN_BPS,MI_MAX_BPS,MR_MEAN_BPS,MR_MEDIAN_BPS,MR_MIN_BPS,MR_MAX_BPS,FWD_TRADES_FOR_MR
str,u32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32
"""VERB""",708281,536025444,0.541075,136.421152,52.06326,64.211393,33.997086,0.0,9983.981312,0.0,-0.0,-0.0,-0.0,-56.28771,-4.004271,-6.1946e6,51917.366786,10
"""FSR""",155584,308787285,0.173557,54.212612,141.000778,84.190576,6.22665,0.0,8148.492743,0.0,0.0,0.0,0.0,-57.566389,-2.476167,-43793.793794,43446.046743,10
"""BMY""",150102,18828776,52.502794,1679.59321,8.06389,10.47267,0.949758,0.0,1330.453564,0.0,0.0,0.0,0.0,-8.337166,-0.946163,-1330.453564,1174.227983,10
